In [1]:
import scipy.ndimage
import numpy as np
dataMatrix = [0] * 10304
for i in range(40):
    for j in range(10):
        imageMatrix = scipy.ndimage.imread('E:/TERM/TERM8/Pattern Recognition/Assignments/Assignments1/orl_faces/s' + str(i+1)+'/' +
                                 str(j+1) +'.pgm',flatten=False, mode=None)
        imagePixelsVector = np.asarray(imageMatrix).reshape(-1)
        dataMatrix = np.vstack([dataMatrix, imagePixelsVector])

In [2]:
#dataMatrix = dataMatrix[1:,0:]
#print(len(dataMatrix[:,0]))

400


In [3]:
dataMatrix

array([[ 48,  49,  45, ...,  47,  46,  46],
       [ 60,  60,  62, ...,  32,  34,  34],
       [ 39,  44,  53, ...,  29,  26,  29],
       ..., 
       [125, 119, 124, ...,  36,  39,  40],
       [119, 120, 120, ...,  89,  94,  85],
       [125, 124, 124, ...,  36,  35,  34]])

In [4]:
labelVector = []
for i in range(40):
    for j in range (10):
        labelVector.append(i+1)
print(len(labelVector))

400


In [101]:
trainingData = dataMatrix[0]
testData = dataMatrix[1]
trainingDataLabelVector = []
testDataLabelVector = []

trainingDataLabelVector.append(labelVector[0])
testDataLabelVector.append(labelVector[1])
index = 2 
while(index < 400):
    if(index % 2 == 0):
        trainingData = np.vstack([trainingData, dataMatrix[index]])
        trainingDataLabelVector.append(labelVector[index])
    else:
         testData = np.vstack([testData, dataMatrix[index]])
         testDataLabelVector.append(labelVector[index])
    index += 1

In [6]:
trainingData

array([[ 48,  49,  45, ...,  47,  46,  46],
       [ 39,  44,  53, ...,  29,  26,  29],
       [ 64,  76,  80, ...,  35,  37,  39],
       ..., 
       [128, 125, 125, ...,  85,  90,  84],
       [129, 127, 133, ...,  93,  93,  93],
       [119, 120, 120, ...,  89,  94,  85]])

In [14]:
meanVector = np.mean(trainingData,axis=0)
meanVector
#print(len(meanVector))

array([ 86.115,  86.265,  86.685, ...,  76.5  ,  77.395,  77.085])

In [15]:
centeredTrainingData = trainingData - meanVector
centeredTrainingData

array([[-38.115, -37.265, -41.685, ..., -29.5  , -31.395, -31.085],
       [-47.115, -42.265, -33.685, ..., -47.5  , -51.395, -48.085],
       [-22.115, -10.265,  -6.685, ..., -41.5  , -40.395, -38.085],
       ..., 
       [ 41.885,  38.735,  38.315, ...,   8.5  ,  12.605,   6.915],
       [ 42.885,  40.735,  46.315, ...,  16.5  ,  15.605,  15.915],
       [ 32.885,  33.735,  33.315, ...,  12.5  ,  16.605,   7.915]])

In [17]:
covMatrix = np.cov(centeredTrainingData.T, bias =True)
covMatrix

array([[ 1241.511775,  1237.489525,  1220.456225, ...,   -69.8075  ,
          -99.755425,   -43.884775],
       [ 1237.489525,  1250.244775,  1227.058475, ...,   -81.5325  ,
         -110.499675,   -55.222525],
       [ 1220.456225,  1227.058475,  1222.145775, ...,   -79.0825  ,
         -114.060575,   -67.653225],
       ..., 
       [  -69.8075  ,   -81.5325  ,   -79.0825  , ...,  1894.      ,
         1865.0575  ,  1682.5425  ],
       [  -99.755425,  -110.499675,  -114.060575, ...,  1865.0575  ,
         1970.238975,  1809.101425],
       [  -43.884775,   -55.222525,   -67.653225, ...,  1682.5425  ,
         1809.101425,  1888.737775]])

In [19]:
eigenvalues, eigenvectors = np.linalg.eigh(covMatrix)

eigenvalues
[ -5.54795470e-10  -4.81218198e-10  -4.23120631e-10 ...,   1.14824189e+06
   2.01725036e+06   2.89336806e+06]
 
eigenvectors
[[ 0.          0.          0.         ...,  0.02055771  0.01361156
  -0.00281623]
 [-0.54555636  0.0476626   0.27946333 ...,  0.02051844  0.01372372
  -0.00278538]
 [ 0.00899758  0.00788202  0.0160391  ...,  0.02030213  0.01375542
  -0.00255135]
 ..., 
 [ 0.00714923  0.00300581 -0.00618886 ...,  0.01558413 -0.01151936
  -0.0054346 ]
 [ 0.00292613 -0.00216155  0.00639018 ...,  0.01481205 -0.01183526
  -0.00555782]
 [ 0.00788795 -0.00268569 -0.0008333  ...,  0.0152495  -0.01087738
  -0.00628804]]


In [24]:
#print('eigenvalues')
#print (eigenvalues)
#print(' ')
#print('eigenvectors')
#print (eigenvectors)
#print(len(eigenvalues))
#print(len(eigenvectors[:,0]))

10304


In [26]:
totalVariance = covMatrix.trace()
totalVariance

16008954.48435

In [141]:
def getPCAAccuracy(alpha):
    eigenvaluesSum = 0
    eigenvaluesIndex = len(eigenvalues) - 1
    while((eigenvaluesSum / totalVariance) < alpha):
        eigenvaluesSum += eigenvalues[eigenvaluesIndex]
        eigenvaluesIndex -= 1
    
    
    #eigenvaluesIndex
    projectionMatrix = eigenvectors[:,eigenvaluesIndex:10303]
    projectionMatrix
    #print(len(projectionMatrix[:,0]))


    ProjectedTrainingDataMatrix = np.matmul(centeredTrainingData, projectionMatrix)
    centeredTestData = testData - meanVector
    ProjectedTestDataMatrix = np.matmul(centeredTestData, projectionMatrix)

    #print(len(ProjectedTrainingDataMatrix[:,0]))
    #print('')
    #print(ProjectedTestDataMatrix[0])


    from sklearn.neighbors import KDTree
    tree = KDTree(ProjectedTrainingDataMatrix, leaf_size=10)



    dist, firstNNindexs = tree.query(ProjectedTestDataMatrix, k=1)
    hits = 0
    for i in range(200):
        #print('test instance label : ',testDataLabelVector[i])
        #print('nearest training instance label : ',trainingDataLabelVector[firstNNindexs[i][0]])
        if(testDataLabelVector[i] == trainingDataLabelVector[firstNNindexs[i][0]]):
            hits += 1
    print('For Alpha = ' + str(alpha) + ', Acuuracy = ' + str((hits/200.0)*100) + '%')

In [142]:
alpha = [0.8, 0.85, 0.9, 0.95]
for i in alpha:
    getPCAAccuracy(i)

For Alpha = 0.8, Acuuracy = 93.5%
For Alpha = 0.85, Acuuracy = 93.0%
For Alpha = 0.9, Acuuracy = 92.0%
For Alpha = 0.95, Acuuracy = 91.5%


The greater the alpha the less accuracy